In [26]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split
import albumentations as alb
import cv2


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
!pip install pydicom
import pydicom  as pdcm

In [29]:
project_path = "/content/drive/My Drive/"

In [30]:
from zipfile import ZipFile

with ZipFile('/content/drive/My Drive/rsna-pneumonia-detection-challenge.zip', 'r') as zipdata:
    data = zipdata.open('stage_2_train_labels.csv')

In [31]:
import pandas as pd

df = pd.read_csv(data)

In [32]:
print('First five rows of Training set:\n', df.head())
print(df.iloc[0])

First five rows of Training set:
                               patientId      x      y  width  height  Target
0  0004cfab-14fd-4e49-80ba-63a80b6bddd6    NaN    NaN    NaN     NaN       0
1  00313ee0-9eaa-42f4-b0ab-c148ed3241cd    NaN    NaN    NaN     NaN       0
2  00322d4d-1c29-4943-afc9-b6754be640eb    NaN    NaN    NaN     NaN       0
3  003d8fa0-6bf1-40ed-b54c-ac657f8495c5    NaN    NaN    NaN     NaN       0
4  00436515-870c-4b36-a041-de91049b9ab4  264.0  152.0  213.0   379.0       1
patientId    0004cfab-14fd-4e49-80ba-63a80b6bddd6
x                                             NaN
y                                             NaN
width                                         NaN
height                                        NaN
Target                                          0
Name: 0, dtype: object


In [33]:
df.columns

Index(['patientId', 'x', 'y', 'width', 'height', 'Target'], dtype='object')

In [34]:
df.isnull().sum()

patientId        0
x            20672
y            20672
width        20672
height       20672
Target           0
dtype: int64

In [35]:
df.shape[0]

30227

In [36]:
df.shape[1]

6

In [37]:
# Number of duplicates in patientId:
print('Number of unique patientId are: {}'.format(df['patientId'].nunique()))

Number of unique patientId are: 26684


In [41]:
df = df.drop_duplicates('patientId')

In [42]:
df.shape[0]

26684

## Distribution of Patients with and without Pneumonia

#### Patients with Pneumonia

In [56]:
## Patients without Pneumonia
wtPnm = df[df.Target == 1]
wtPnm 

,patientId,x,y,width,height,Target
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1
14,00aecb01-a116-45a2-956c-08d2fa55433f,288.0,322.0,94.0,135.0,1
16,00c0b293-48e7-4e16-ac76-9269ba535a62,306.0,544.0,168.0,244.0,1
19,00f08de1-517e-4652-a04f-d1dc9ee48593,181.0,184.0,206.0,506.0,1
...,...,...,...,...,...,...
30208,c18d1138-ba74-4af5-af21-bdd4d2c96bb5,646.0,386.0,141.0,206.0,1
30214,c1cddf32-b957-4753-acaa-472ab1447e86,269.0,416.0,193.0,285.0,1
30219,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,666.0,418.0,186.0,223.0,1
30221,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,609.0,464.0,240.0,284.0,1


#### Patients without Pneumonia

In [57]:
## Patients without Pneumonia
WtoPnm = df[df.Target == 0]
WtoPnm

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
6,00569f44-917d-4c86-a842-81832af98c30,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...
30216,c1cf3255-d734-4980-bfe0-967902ad7ed9,NaN,NaN,NaN,NaN,0
30217,c1e228e4-b7b4-432b-a735-36c48fdb806f,NaN,NaN,NaN,NaN,0
30218,c1e3eb82-c55a-471f-a57f-fe1a823469da,NaN,NaN,NaN,NaN,0
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0


In [58]:
print(f'Percentage of Patients with Pneumonia :  {round(wtPnm.shape[0]/df.shape[0]*100, 0)}% ')
print(f'Percentage of Patients without Pneumonia  {round(WtoPnm.shape[0]/df.shape[0]*100, 0)}%')

Percentage of Patients with Pneumonia :  23.0% 
Percentage of Patients without Pneumonia  77.0%
